### Chuẩn bị data

```text
datasets/
├── BEAT/
│   ├── 1/
│   ├── 2/
|   ├── ...
|   └── ...
├── BEAT_numpy/
│   ├── npy/
│   └── txt/
```

In [ ]:
base_dir = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT"
out_dir = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT_numpy"

### Tính mean/std cho toàn bộ dataset

In [ ]:
# !python "/home/serverai/ltdoanh/Motion_Diffusion/datasets/step1_fit_scaler.py" --parent-dir "{base_dir}" --start 1 --end 30 --mode hybrid

### Chuyển dataset bvh sang npy theo từng segment

In [ ]:
# import os
# for i in range(6, 31):
#     print("Đang chạy preprocess_data.py...")
#     command2 = f'python "/home/serverai/ltdoanh/Motion_Diffusion/datasets/preprocess_data.py" --parent-dir "{base_dir}" --out-root "{out_dir}" --folders "{i}"'
#     os.system(command2)

In [ ]:
# !python "/home/serverai/ltdoanh/Motion_Diffusion/datasets/preprocess_data.py" --parent-dir "{base_dir}" --out-root "{out_dir}" --folders "5"

In [ ]:
# from textgrid import TextGrid

# tg = TextGrid.fromFile("/home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT/5/5_stewart_0_82_82.TextGrid")
# print(f"Number of tiers: {len(tg)}")
# for i, tier in enumerate(tg):
#     print(f"Tier {i}: {tier.name}, intervals: {len(tier.intervals)}")

### Train MotionDiffuse như bình thường

In [ ]:
# !python "/home/serverai/ltdoanh/Motion_Diffusion/tools/train.py" --dataset_name beat

### Train VQ-VAE

In [ ]:
!python "/home/serverai/ltdoanh/Motion_Diffusion/tools/train_vq.py" --dataset_name beat --codebook_size 512

In [ ]:
!python ./tools/scale_factor.py --data_root ./datasets/BEAT_numpy --checkpoints_dir ./checkpoints --vqvae_name VQVAE_BEAT --batch_size 32 --max_samples 5000

### Train MotionDiffuse trên Latent Space do VQ-VAE ở trên

In [ ]:
# !python "/home/serverai/ltdoanh/Motion_Diffusion/tools/train_vq_diffusion.py" --dataset_name beat --vqvae_name VQVAE_BEAT --sampler ddim --max_epoch 200

In [ ]:
!python ./tools/train_vq_diffusion.py --dataset_name beat --data_root ./datasets/BEAT_numpy --vqvae_name VQVAE_BEAT --scale_factor 7.53435087 --batch_size 64 --max_epoch 100 --cond_drop_prob 0.1 --guidance_scale 2.0 --hand_loss_weight 10.0 --lr 2e-4

### Evaluation - Đang fix lỗi ...

In [ ]:
# !python "/home/serverai/ltdoanh/Motion_Diffusion/run_evaluation.py"

### Visual data bằng mean/std chuẩn tính từ bộ dữ liệu

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import sys
import os
import numpy as np
import joblib

# Setup path
PYOM_DIR = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/pymo"
if PYOM_DIR not in sys.path:
    sys.path.insert(0, PYOM_DIR)

from pymo.parsers import BVHParser
from pymo.preprocessing import *
from pymo.viz_tools import draw_stickfigure, draw_stickfigure3d
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

print("✅ Imports successful!")

# ===== CELL 2: Load Pipeline =====
pipeline_path = "/home/serverai/ltdoanh/Motion_Diffusion/global_pipeline.pkl"

print(f"📦 Loading pipeline from: {pipeline_path}")

pipeline = joblib.load(pipeline_path)
print(f"✅ Pipeline loaded!")
print(f"   Mean shape: {pipeline.named_steps['stdscale'].data_mean_.shape}, First 5 values: {pipeline.named_steps['stdscale'].data_mean_[:5]}")
print(f"   Std shape: {pipeline.named_steps['stdscale'].data_std_.shape},  First 5 values: {pipeline.named_steps['stdscale'].data_std_[:5]}")

# ===== CELL 3: Load and Visualize Motion =====
npy_path = "/home/serverai/ltdoanh/Motion_Diffusion/results/motion_inference-02.npy"
frame = 50

print(f"\n🎬 Visualizing: {os.path.basename(npy_path)}")
print(f"   Frame: {frame}")

# Load motion data
motion_data = np.load(npy_path)
print(f"   Motion shape: {motion_data.shape}")

# Inverse transform
print("   Performing inverse transform...")
reconstructed = pipeline.inverse_transform([motion_data])
print(f"   ✅ Reconstructed shape: {reconstructed[0].values.shape}")

# Visualize 2D
print("\n   Creating 2D visualization...")
fig1 = plt.figure(figsize=(10, 8))
draw_stickfigure(reconstructed[0], frame=frame)
plt.title(f"2D Stick Figure - Frame {frame}")
plt.tight_layout()
plt.show()

print("   ✅ 2D plot displayed!")

# Visualize 3D
print("\n   Creating 3D visualization...")
fig2 = plt.figure(figsize=(10, 8))
draw_stickfigure3d(reconstructed[0], frame=frame)
plt.title(f"3D Stick Figure - Frame {frame}")
plt.tight_layout()
plt.show()

print("   ✅ 3D plot displayed!")

### Visual data bằng mean/std từ model được huấn luyện

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import numpy as np
import joblib
import torch

# Setup path
PYOM_DIR = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/pymo"
if PYOM_DIR not in sys.path:
    sys.path.insert(0, PYOM_DIR)

from pymo.parsers import BVHParser
from pymo.preprocessing import *
from pymo.viz_tools import draw_stickfigure, draw_stickfigure3d
from sklearn.pipeline import Pipeline

print("✅ Imports successful!")

# ===== CREATE PIPELINE WITH META =====
print("\n🔨 Creating pipeline from scratch...")

# 1. Create empty pipeline
pipeline = Pipeline([
    ('param', MocapParameterizer('position')),
    ('rcpn', RootCentricPositionNormalizer()),
    ('delta', RootTransformer('abdolute_translation_deltas')),
    ('const', ConstantsRemover()),
    ('np', Numpyfier()),
    ('down', DownSampler(2)),
    ('stdscale', ListStandardScaler())
])

# 2. Fit pipeline on sample BVH (to learn structure)
bvh_sample_path = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT/1/1_wayne_0_1_1.bvh"
print(f"   Fitting on BVH sample: {os.path.basename(bvh_sample_path)}")

parser = BVHParser()
parsed_data = parser.parse(bvh_sample_path)
pipeline.fit([parsed_data])
print("   ✅ Pipeline fitted (structure learned)")

# 3. Load mean/std from meta directory
# meta_dir = "/home/serverai/ltdoanh/Motion_Diffusion/checkpoints/beat/test/meta"
# mean_path = os.path.join(meta_dir, "mean.npy")
# std_path = os.path.join(meta_dir, "std.npy")
ckpt_path = "/home/serverai/ltdoanh/Motion_Diffusion/checkpoints/beat/vq_diffusion/model/latest.pt"  # Thay đường dẫn file của bạn vào đây

try:
    # 2. Load checkpoint
    # map_location='cpu' giúp tránh lỗi nếu máy bạn không có GPU giống lúc train
    checkpoint = torch.load(ckpt_path, map_location='cpu', weights_only=False)

    # 3. Kiểm tra và lấy mean, std
    if 'mean' in checkpoint and 'std' in checkpoint:
        mean = checkpoint['mean']
        std = checkpoint['std']

        print("--- Đã tìm thấy Mean và Std ---")
        print(f"Shape của Mean: {mean.shape}")
        print(f"Shape của Std: {std.shape}")
        
        # In thử vài giá trị đầu
        print(f"Mean (5 giá trị đầu): {mean[:5]}")
        print(f"Std (5 giá trị đầu): {std[:5]}")
        
        # 4. (Tùy chọn) Lưu lại ra file .npy để dùng việc khác nếu cần
        # np.save('mean.npy', mean)
        # np.save('std.npy', std)
        # print("Đã lưu ra file .npy")
        
    else:
        print("Không tìm thấy key 'mean' hoặc 'std' trong file .pt này.")
        print("Các keys hiện có:", checkpoint.keys())

except Exception as e:
    print(f"Có lỗi xảy ra: {e}")

# 4. Override pipeline's mean/std with meta values
print("\n🔧 Overriding pipeline statistics with meta values...")
pipeline.named_steps['stdscale'].data_mean_ = mean
pipeline.named_steps['stdscale'].data_std_ = std
print("   ✅ Pipeline updated with meta statistics!")

# ===== VISUALIZE MOTION =====
npy_path = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT_numpy/npy/2/2_scott_0_2_2_sentence_000.npy"
frame = 50

print(f"\n🎬 Visualizing: {os.path.basename(npy_path)}")
print(f"   Frame: {frame}")

# Load motion data
motion_data = np.load(npy_path)
print(f"   Motion shape: {motion_data.shape}")

# Inverse transform
print("   Performing inverse transform...")
reconstructed = pipeline.inverse_transform([motion_data])
print(f"   ✅ Reconstructed shape: {reconstructed[0].values.shape}")

# Visualize 2D
print("\n   Creating 2D visualization...")
fig1 = plt.figure(figsize=(10, 8))
draw_stickfigure(reconstructed[0], frame=frame)
plt.title(f"2D Stick Figure (Meta) - Frame {frame}")
plt.tight_layout()
plt.show()

print("   ✅ 2D plot displayed!")

# Visualize 3D
print("\n   Creating 3D visualization...")
fig2 = plt.figure(figsize=(10, 8))
draw_stickfigure3d(reconstructed[0], frame=frame)
plt.title(f"3D Stick Figure (Meta) - Frame {frame}")
plt.tight_layout()
plt.show()

print("   ✅ 3D plot displayed!")

### Inference 

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import os
import sys

from models import MotionTransformer
from trainers import DDPMTrainer
from models.vq.model import RVQVAE 

# ==========================================
# 1. Cấu hình Inference
# ==========================================
class InferenceConfig:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

        self.is_train = False
        self.schedule_sampler = 'uniform'   

        # --- Cấu hình Diffusion ---
        self.input_feats = 512     # Latent Dimension
        self.num_frames = 45      # Latent Length (360 / 8)
        self.num_layers = 8
        self.latent_dim = 512
        self.ff_size = 1024
        self.num_heads = 8
        self.dropout = 0.1
        self.activation = "gelu"
        self.dataset_name = 'beat' 
        self.do_denoise = True
        self.noise_schedule = 'cosine'
        self.diffusion_steps = 1000
        self.no_clip = False
        self.no_eff = False
        self.result_dir = "/home/serverai/ltdoanh/Motion_Diffusion/results"

# Class giả lập args cho RVQVAE
class VQArgs:
    def __init__(self):
        # Các giá trị mặc định, sẽ được cập nhật logic bên dưới
        self.num_quantizers = 1 
        self.shared_codebook = False
        self.quantize_dropout_prob = 0.0
        self.mu = 0.99 # Cho QuantizerEMA

opt = InferenceConfig()
vq_args = VQArgs()

# ==========================================
# 2. Load Checkpoint & Tách Weights
# ==========================================
ckpt_path = "/home/serverai/ltdoanh/Motion_Diffusion/checkpoints/beat/vq_diffusion/model/best_model.pt"
print(f"📂 Loading checkpoint: {ckpt_path}")

# Load toàn bộ checkpoint
checkpoint = torch.load(ckpt_path, map_location='cpu', weights_only=False)
state_dict = checkpoint['model_state_dict']

# --- Tự động phát hiện số lượng Quantizers từ Checkpoint ---
# Điều này giúp tránh lỗi sai lệch key khi khởi tạo VQ-VAE
max_layer_idx = 0
for k in state_dict.keys():
    if "vqvae.quantizer.layers." in k:
        # Parse tìm số lớn nhất trong 'layers.X.'
        try:
            parts = k.split('.')
            layer_idx = int(parts[parts.index('layers') + 1])
            if layer_idx > max_layer_idx:
                max_layer_idx = layer_idx
        except:
            pass

vq_args.num_quantizers = max_layer_idx + 1
print(f"🔍 Detected num_quantizers: {vq_args.num_quantizers}")

# --- Tách Dictionary ---
trans_dict = {}
vqvae_dict = {}

for k, v in state_dict.items():
    if k.startswith('transformer.'):
        trans_dict[k[12:]] = v  
    elif k.startswith('vqvae.'):
        vqvae_dict[k[6:]] = v   

# ==========================================
# 3. Khởi tạo Models
# ==========================================

# A. Motion Transformer
print("🔧 Initializing MotionTransformer...")
encoder = MotionTransformer(
    input_feats=opt.input_feats,
    num_frames=opt.num_frames,
    num_layers=opt.num_layers,
    latent_dim=opt.latent_dim,
    num_heads=opt.num_heads,
    ff_size=opt.ff_size,
    no_clip=opt.no_clip,
    no_eff=opt.no_eff
)
encoder.load_state_dict(trans_dict, strict=True)
encoder.to(opt.device).eval()

# B. RVQVAE
print("🔧 Initializing RVQVAE...")
# Lưu ý: Các tham số dưới đây phải khớp với file config lúc train VQVAE của bạn.
# Tôi đang để các giá trị phổ biến dựa trên file model.py
vqvae_model = RVQVAE(
    args=vq_args,
    input_width=264,       # BEAT dataset thường là 264
    nb_code=512,           # Kiểm tra lại config train cũ nếu lỗi
    code_dim=512, 
    output_emb_width=512, 
    down_t=3, 
    stride_t=2, 
    width=512, 
    depth=3, 
    dilation_growth_rate=3,
    activation='relu',
    norm=None
)
vqvae_model.load_state_dict(vqvae_dict, strict=True)
vqvae_model.to(opt.device).eval()

# ==========================================
# 4. Inference
# ==========================================
trainer = DDPMTrainer(opt, encoder)

# Inject mean/std (Quan trọng cho quá trình decode cuối cùng)
trainer.mean = checkpoint['mean']
trainer.std = checkpoint['std']

print("🚀 Starting Inference...")
os.makedirs(opt.result_dir, exist_ok=True)

with torch.no_grad():
    caption = ["the first thing i like to do on weekends is relaxing and i'll go shopping if i'm not that tired"]
    
    # Độ dài Latent (45)
    m_lens = torch.LongTensor([45]).to(opt.device) 
    
    # 1. Sinh Latent (Diffusion) -> Output: (Batch, Length, Dim) = (1, 45, 512)
    pred_latent_list = trainer.generate(caption, m_lens, dim_pose=512)
    pred_latent = pred_latent_list[0]

    if pred_latent.dim() == 2:
        pred_latent = pred_latent.unsqueeze(0)  # Thêm batch dim nếu cần

    print(f"   Latent generated shape: {pred_latent.shape}")

    # 2. Decode bằng RVQVAE
    # RVQVAE Decoder cần input: (Batch, Channel, Length) -> Cần permute
    latent_input = pred_latent.permute(0, 2, 1) # -> (1, 512, 45)
    
    print("   Decoding with RVQVAE...")
    # Gọi trực tiếp decoder (bỏ qua quantizer vì Diffusion đã sinh ra latent rồi)
    decoded_motion = vqvae_model.decoder(latent_input)
    
    # 3. Post-process (Permute lại về: Batch, Length, Channel)
    # Hàm postprocess trong model.py: (B, C, T) -> (B, T, C)
    motion = vqvae_model.postprocess(decoded_motion).cpu().numpy()

    if motion.shape[1] == 264 and motion.shape[2] == 360:
        motion = motion.transpose(0, 2, 1)

    # motion = motion.cpu().numpy()
    
    # 4. Denormalize (Giải chuẩn hóa)
    # Output của VQVAE thường vẫn là normalized data
    mean = checkpoint['mean']
    std = checkpoint['std']
    
    # Đảm bảo shape khớp để broadcast
    # Motion: (1, 360, 264), Mean: (264,), Std: (264,)
    motion = motion * std + mean
    
    print(f"🎉 Final Motion Shape: {motion.shape}")

# Lưu kết quả
save_path = os.path.join(opt.result_dir, 'inference-latest.npy')
np.save(save_path, motion)
print(f"💾 Saved to: {save_path}")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import numpy as np
import joblib
import torch

# Setup path
PYOM_DIR = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/pymo"
if PYOM_DIR not in sys.path:
    sys.path.insert(0, PYOM_DIR)

from pymo.parsers import BVHParser
from pymo.preprocessing import *
from pymo.viz_tools import draw_stickfigure, draw_stickfigure3d
from sklearn.pipeline import Pipeline

print("✅ Imports successful!")

# ===== CREATE PIPELINE WITH META =====
print("\n🔨 Creating pipeline from scratch...")

# 1. Create empty pipeline
pipeline = Pipeline([
    ('param', MocapParameterizer('position')),
    ('rcpn', RootCentricPositionNormalizer()),
    ('delta', RootTransformer('abdolute_translation_deltas')),
    ('const', ConstantsRemover()),
    ('np', Numpyfier()),
    ('down', DownSampler(2)),
    ('stdscale', ListStandardScaler())
])

# 2. Fit pipeline on sample BVH (to learn structure)
bvh_sample_path = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT/1/1_wayne_0_1_1.bvh"
print(f"   Fitting on BVH sample: {os.path.basename(bvh_sample_path)}")

parser = BVHParser()
parsed_data = parser.parse(bvh_sample_path)
pipeline.fit([parsed_data])
print("   ✅ Pipeline fitted (structure learned)")

# 3. Load mean/std from meta directory
ckpt_path = "/home/serverai/ltdoanh/Motion_Diffusion/checkpoints/beat/vq_diffusion/model/best_model.pt"  

try:
    # 2. Load checkpoint
    # map_location='cpu' giúp tránh lỗi nếu máy bạn không có GPU giống lúc train
    checkpoint = torch.load(ckpt_path, map_location='cpu', weights_only=False)

    # 3. Kiểm tra và lấy mean, std
    if 'mean' in checkpoint and 'std' in checkpoint:
        mean = checkpoint['mean']
        std = checkpoint['std']

        print("--- Đã tìm thấy Mean và Std ---")
        print(f"Shape của Mean: {mean.shape}")
        print(f"Shape của Std: {std.shape}")
        
        # In thử vài giá trị đầu
        print(f"Mean (5 giá trị đầu): {mean[:5]}")
        print(f"Std (5 giá trị đầu): {std[:5]}")
        
    else:
        print("Không tìm thấy key 'mean' hoặc 'std' trong file .pt này.")
        print("Các keys hiện có:", checkpoint.keys())

except Exception as e:
    print(f"Có lỗi xảy ra: {e}")

# 4. Override pipeline's mean/std with meta values
print("\n🔧 Overriding pipeline statistics with meta values...")
pipeline.named_steps['stdscale'].data_mean_ = mean
pipeline.named_steps['stdscale'].data_std_ = std
print("   ✅ Pipeline updated with meta statistics!")

# ===== VISUALIZE MOTION =====
npy_path = "/home/serverai/ltdoanh/Motion_Diffusion/results/inference-latest.npy"
frame = 50

print(f"\n🎬 Visualizing: {os.path.basename(npy_path)}")
print(f"   Frame: {frame}")

# Load motion data
motion_data = np.load(npy_path)
print(f"   Motion shape: {motion_data.shape}")
if motion_data.ndim == 3:
    motion_data = motion_data[0] # Lấy mẫu đầu tiên -> (360, 264)
    print(f"   Squeezed Motion shape: {motion_data.shape}")

# Inverse transform
print("   Performing inverse transform...")
reconstructed = pipeline.inverse_transform([motion_data])
print(f"   ✅ Reconstructed shape: {reconstructed[0].values.shape}")

# Visualize 2D
print("\n   Creating 2D visualization...")
fig1 = plt.figure(figsize=(10, 8))
draw_stickfigure(reconstructed[0], frame=frame)
plt.title(f"2D Stick Figure (Meta) - Frame {frame}")
plt.tight_layout()
plt.show()

print("   ✅ 2D plot displayed!")

# Visualize 3D
print("\n   Creating 3D visualization...")
fig2 = plt.figure(figsize=(10, 8))
draw_stickfigure3d(reconstructed[0], frame=frame)
plt.title(f"3D Stick Figure (Meta) - Frame {frame}")
plt.tight_layout()
plt.show()

print("   ✅ 3D plot displayed!")

### Inference DDIM

In [ ]:
# ==========================================
# 1. SETUP & IMPORTS
# ==========================================
import os
import sys

# Add project root to path
ROOT = "/home/serverai/ltdoanh/Motion_Diffusion"
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

PYMO_DIR = os.path.join(ROOT, 'datasets', 'pymo')
if PYMO_DIR not in sys.path:
    sys.path.insert(0, PYMO_DIR)

import torch
import numpy as np
import joblib
import matplotlib.pyplot as plt
from IPython.display import Image, display

# ==========================================
# 2. CONFIGURATION
# ==========================================
class Config:
    # Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Paths
    checkpoints_dir = "/home/serverai/ltdoanh/Motion_Diffusion/checkpoints"
    dataset_name = 'beat'
    vqvae_name = 'VQVAE_BEAT'
    diffusion_name = 'vq_diffusion'
    checkpoint_name = 'best_model.pt'
    result_dir = "/home/serverai/ltdoanh/Motion_Diffusion/results"
    scale_factor = 7.53435087  # Scale factor used during training
    
    # Model config
    latent_dim = 512
    num_layers = 8
    num_heads = 8
    ff_size = 1024
    dropout = 0.1
    no_eff = False
    freeze_vqvae = True
    
    # Diffusion
    diffusion_steps = 1000
    noise_schedule = 'cosine'
    
    # Sampling
    sampler = 'ddim'  # 'ddpm' or 'ddim'
    ddim_eta = 0.0
    num_samples = 1

args = Config()

# ==========================================
# 3. LOAD MODEL
# ==========================================
from models.vq_diffusion import create_vq_latent_diffusion, VQLatentDiffusionWrapper
from models.gaussian_diffusion import (
    GaussianDiffusion, get_named_beta_schedule,
    ModelMeanType, ModelVarType, LossType
)

print("\n📦 Loading model...")

# Create model
model = create_vq_latent_diffusion(
    dataset_name=args.dataset_name,
    vqvae_name=args.vqvae_name,
    checkpoints_dir=args.checkpoints_dir,
    device=args.device,
    freeze_vqvae=args.freeze_vqvae,
    scale_factor=args.scale_factor,
    latent_dim=args.latent_dim,
    num_layers=args.num_layers,
    num_heads=args.num_heads,
    ff_size=args.ff_size,
    dropout=args.dropout,
    no_eff=args.no_eff
)

# Load checkpoint
checkpoint_path = os.path.join(
    args.checkpoints_dir,
    args.dataset_name,
    args.diffusion_name,
    'model',
    args.checkpoint_name
)

print(f"Loading checkpoint: {checkpoint_path}")
checkpoint = torch.load(checkpoint_path, map_location='cpu', weights_only=False)

try:
    model.load_state_dict(checkpoint['model_state_dict'], strict=True)
    print("✓ Checkpoint loaded (strict)")
except:
    model.load_state_dict(checkpoint['model_state_dict'], strict=False)
    print("✓ Checkpoint loaded (loose)")

model.eval()

# Create diffusion
betas = get_named_beta_schedule(args.noise_schedule, args.diffusion_steps)
diffusion = GaussianDiffusion(
    betas=betas,
    model_mean_type=ModelMeanType.EPSILON,
    model_var_type=ModelVarType.FIXED_SMALL,
    loss_type=LossType.MSE,
    rescale_timesteps=False
)

print("✓ Model and diffusion ready")

# ==========================================
# 4. LOAD STATS
# ==========================================
stats_path = os.path.join(ROOT, 'global_pipeline.pkl')
pipeline = joblib.load(stats_path)
scaler = pipeline.named_steps['stdscale']
mean = scaler.data_mean_
std = scaler.data_std_

print(f"✓ Stats loaded (dim={len(mean)})")

# ==========================================
# 5. GENERATE MOTION
# ==========================================
@torch.no_grad()
def generate_motion(text_prompts, lengths):
    """Generate motion from text"""
    
    print(f"\n🎬 Generating motion...")
    print(f"Prompts: {text_prompts}")
    
    wrapped_model = VQLatentDiffusionWrapper(model)
    
    B = len(text_prompts)
    T_latent = model.num_frames
    code_dim = model.vqvae.code_dim
    
    shape = (B, T_latent, code_dim)
    
    model_kwargs = {
        'y': {
            'text': text_prompts,
            'length': lengths,
        }
    }
    
    # Sample
    if args.sampler == 'ddim':
        print(f"Sampling with DDIM (eta={args.ddim_eta})...")
        latent_samples = diffusion.ddim_sample_loop(
            wrapped_model, shape,
            clip_denoised=False,
            model_kwargs=model_kwargs,
            device=args.device,
            progress=True,
            eta=args.ddim_eta
        )
    else:
        print("Sampling with DDPM...")
        latent_samples = diffusion.p_sample_loop(
            wrapped_model, shape,
            clip_denoised=False,
            model_kwargs=model_kwargs,
            device=args.device,
            progress=True
        )
    
    print(f"Latent shape: {latent_samples.shape}")
    
    # Decode
    print("Decoding to motion...")
    motion = model.decode_from_latent(latent=latent_samples)
    
    print(f"Motion shape: {motion.shape}")
    
    # Denormalize
    motion_np = motion.cpu().numpy()
    if motion_np.shape[1] == 264:
        motion_np = motion_np.transpose(0, 2, 1)
    
    motion_denorm = motion_np * std + mean
    
    print(f"✓ Generation complete: {motion_denorm.shape}")
    
    return motion_denorm

# ==========================================
# 6. INFERENCE
# ==========================================
# Define your prompts here
text_prompts = [
    "for example people being shifted from the center of the frame to the left side of the frame can make a different video when singing contest with the background"
    # "how are you"
]

# Latent lengths (45 for BEAT = 360 original frames)
lengths = [45] * len(text_prompts)

# Generate!
motion = generate_motion(text_prompts, lengths)

# ==========================================
# 7. SAVE RESULTS
# ==========================================
os.makedirs(args.result_dir, exist_ok=True)

save_path = os.path.join(args.result_dir, 'motion_0005.npy')
np.save(save_path, motion)
print(f"💾 Saved to: {save_path}")

# Save prompts
with open(os.path.join(args.result_dir, 'prompts.txt'), 'w') as f:
    for i, prompt in enumerate(text_prompts):
        f.write(f"{i}: {prompt}\n")

print("\n" + "="*60)
print("INFERENCE COMPLETED!")
print("="*60)
print(f"Motion saved to: {args.result_dir}")
print("\nTo create more visualizations, run:")
print(f"python tools/visualize_motion.py --motion_path {save_path}")
print("="*60)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import numpy as np
import joblib
import pandas as pd

# Setup path
PYOM_DIR = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/pymo"
if PYOM_DIR not in sys.path:
    sys.path.insert(0, PYOM_DIR)

from pymo.parsers import BVHParser
from pymo.viz_tools import draw_stickfigure, draw_stickfigure3d

print("✅ Imports successful!")

# ==========================================
# 1. LOAD & FIX PIPELINE (ADVANCED FIX)
# ==========================================
from sklearn.pipeline import Pipeline

pipeline_path = "/home/serverai/ltdoanh/Motion_Diffusion/global_pipeline.pkl"
print(f"📦 Loading pipeline from: {pipeline_path}")
pipeline = joblib.load(pipeline_path)

# Load Reference BVH
ref_bvh_path = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT/1/1_wayne_0_1_1.bvh"

if os.path.exists(ref_bvh_path):
    print(f"🔧 Processing reference BVH: {os.path.basename(ref_bvh_path)}")
    parser = BVHParser()
    ref_data = parser.parse(ref_bvh_path)
    
    # [QUAN TRỌNG] Tìm vị trí của bước 'np' (Numpyfier)
    # Chúng ta cần chạy dữ liệu qua tất cả các bước TRƯỚC bước 'np'
    # để biến đổi dữ liệu thô (228) thành dữ liệu model (264)
    
    steps_before_np = []
    np_found = False
    
    for name, step in pipeline.steps:
        if name == 'np': # Tên của Numpyfier trong pipeline
            np_found = True
            break
        steps_before_np.append((name, step))
    
    if np_found and steps_before_np:
        print(f"   Running reference data through pre-processing steps: {[n for n, _ in steps_before_np]}...")
        
        # Tạo pipeline con chỉ chứa các bước tiền xử lý
        pre_pipeline = Pipeline(steps_before_np)
        
        # Biến đổi reference data: (Time, 228) -> (Time, 264)
        # fit_transform cần list các MocapData
        processed_ref = pre_pipeline.fit_transform([ref_data])
        
        # Lấy kết quả (đã là DataFrame đúng chuẩn 264 cột)
        template_data = processed_ref[0]
        
        print(f"   Template shape after preprocessing: {template_data.values.shape}") 
        # Kỳ vọng output: (Time, 264)
        
        if template_data.values.shape[1] == 264:
            # Gán template đúng shape vào Numpyfier
            pipeline.named_steps['np'].org_mocap_ = template_data
            print("✅ Skeleton structure injected with CORRECT dimensions (264)!")
        else:
            print(f"⚠️ Warning: Processed shape is {template_data.values.shape[1]}, but model output is 264.")
            print("   Visualization might still fail.")
            
    else:
        print("⚠️ Could not isolate steps before Numpyfier.")
        # Fallback cũ (sẽ lỗi)
        pipeline.named_steps['np'].org_mocap_ = ref_data

else:
    print(f"❌ Reference BVH not found at {ref_bvh_path}")
    sys.exit(1)

# ==========================================
# 2. LOAD & PREPARE DATA
# ==========================================
npy_path = "/home/serverai/ltdoanh/Motion_Diffusion/results/generated_motion_best_model.npy"
print(f"\n📂 Loading motion: {npy_path}")
motion_data = np.load(npy_path)

# Xử lý dimension: (1, 360, 264) -> (360, 264)
if motion_data.ndim == 3:
    motion_data = motion_data[0]

print(f"   Raw Input Shape: {motion_data.shape}")
print(f"   Raw Input Range: {motion_data.min():.2f} to {motion_data.max():.2f}")

# [FIX 2] Xử lý vấn đề "Giá trị khổng lồ" (9 triệu)
# Nếu giá trị > 1000, chắc chắn file npy này ĐÃ LÀ real-world scale (đã denormalize).
# Nhưng pipeline.inverse_transform() lại mong đợi input là Normalized (Mean~0, Std~1).
# Ta phải "Re-normalize" thủ công bằng Mean/Std của pipeline.

scaler = pipeline.named_steps['stdscale']
mean = scaler.data_mean_
std = scaler.data_std_

# Kiểm tra xem có cần re-normalize không
if np.abs(motion_data).max() > 100:
    print("⚠️ Detected large values (Real-scale data). Re-normalizing for pipeline compatibility...")
    # Công thức: (X - Mean) / Std
    motion_data = (motion_data - mean) / std
    print(f"   Normalized Range: {motion_data.min():.2f} to {motion_data.max():.2f}")
else:
    print("✅ Data seems to be already normalized.")

# ==========================================
# 3. INVERSE TRANSFORM & VISUALIZE
# ==========================================
print("\n🔄 Performing inverse transform...")
try:
    # Inverse transform trả về list các MocapData
    reconstructed = pipeline.inverse_transform([motion_data])
    mocap_data = reconstructed[0]
    
    print("   ✅ Inverse transform successful!")
    
    # Kiểm tra giá trị sau khi tái tạo (phải tính bằng cm/m)
    df = mocap_data.values
    print(f"   Final Motion Range (cm): {df.min().min():.2f} to {df.max().max():.2f}")
    
    # Visualize
    frame = 0
    print(f"\n🎬 Visualizing Frame {frame}...")
    
    plt.figure(figsize=(10, 8))
    draw_stickfigure(mocap_data, frame=frame)
    plt.gca().invert_yaxis()
    plt.title(f"Reconstructed Motion - Frame {frame}")
    plt.show()
    
except Exception as e:
    print(f"\n❌ ERROR during visualization: {e}")
    import traceback
    traceback.print_exc()

In [26]:
import os
import sys
import numpy as np
import joblib
import pandas as pd
from sklearn.pipeline import Pipeline

# # 1. Cấu hình đường dẫn
# ROOT = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# if ROOT not in sys.path:
#     sys.path.insert(0, ROOT)

PYMO_DIR = os.path.join(ROOT, 'datasets', 'pymo')
if PYMO_DIR not in sys.path:
    sys.path.insert(0, PYMO_DIR)

from pymo.parsers import BVHParser

# Đường dẫn (Sửa lại nếu cần)
pipeline_path = os.path.join(ROOT, "global_pipeline.pkl")
ref_bvh_path = os.path.join(ROOT, "datasets/BEAT/1/1_wayne_0_1_1.bvh")
save_dir = os.path.join(ROOT, "datasets/BEAT_indices")

print(f"🚀 Bắt đầu tạo indices...")

# 2. Load Pipeline & Lấy tên cột
if not os.path.exists(ref_bvh_path):
    print(f"❌ Lỗi: Không tìm thấy file BVH mẫu tại {ref_bvh_path}")
    sys.exit(1)

pipeline = joblib.load(pipeline_path)
parser = BVHParser()
ref_data = parser.parse(ref_bvh_path)

# Tìm bước 'np' (Numpyfier)
np_step_idx = -1
for i, (name, step) in enumerate(pipeline.steps):
    if name == 'np':
        np_step_idx = i
        break

# Chạy pipeline con để fit dữ liệu (Bắt buộc chạy để cập nhật trạng thái bên trong)
print("⚙️ Đang trích xuất tên features...")
sub_pipeline = Pipeline(pipeline.steps[:np_step_idx+1])

# Ta vẫn chạy fit_transform, nhưng KHÔNG dùng kết quả trả về để lấy tên cột
_ = sub_pipeline.fit_transform([ref_data]) 

# [FIX] Lấy tên cột từ thuộc tính nội bộ của bước Numpyfier
try:
    # Truy cập vào bước 'np' trong pipeline đã fit
    numpyfier_step = sub_pipeline.named_steps['np']
    
    # Kiểm tra xem org_mocap_ lưu DataFrame trực tiếp hay bọc trong MocapData
    if hasattr(numpyfier_step.org_mocap_, 'columns'):
        # Trường hợp 1: org_mocap_ là DataFrame
        feature_names = list(numpyfier_step.org_mocap_.columns)
    elif hasattr(numpyfier_step.org_mocap_, 'values'):
        # Trường hợp 2: org_mocap_ là MocapData (pymo thường dùng cách này)
        feature_names = list(numpyfier_step.org_mocap_.values.columns)
    else:
        raise ValueError("Không tìm thấy thuộc tính columns trong org_mocap_")

    print(f"✅ Tìm thấy {len(feature_names)} features (Khớp 264).")
    
    # In thử 5 tên đầu tiên để kiểm tra
    print(f"   Sample features: {feature_names[:5]}")

except Exception as e:
    print(f"❌ Lỗi khi trích xuất từ Numpyfier: {e}")
    sys.exit(1)

# 3. Logic tách Hand vs Body (Giữ nguyên)
hand_keywords = ['Hand', 'Finger', 'Thumb', 'Index', 'Middle', 'Ring', 'Pinky', 'Carpal']

hand_indices = []
body_indices = []

print("\n🔍 Đang phân loại...")
for idx, name in enumerate(feature_names):
    # Nếu tên khớp chứa từ khóa tay
    is_hand = any(k.lower() in name.lower() for k in hand_keywords)
    # Loại trừ cánh tay/vai (ForeArm, Arm, Shoulder thuộc về Body)
    if 'ForeArm' in name or 'Arm' in name or 'Shoulder' in name:
        is_hand = False
        
    if is_hand:
        hand_indices.append(idx)
    else:
        body_indices.append(idx)

# 4. Lưu file (Giữ nguyên)
os.makedirs(save_dir, exist_ok=True)
np.save(os.path.join(save_dir, 'hand_indices.npy'), np.array(hand_indices))
np.save(os.path.join(save_dir, 'body_indices.npy'), np.array(body_indices))

print(f"\n💾 Đã lưu thành công:")
print(f"   - {os.path.join(save_dir, 'hand_indices.npy')} ({len(hand_indices)} chiều)")
print(f"   - {os.path.join(save_dir, 'body_indices.npy')} ({len(body_indices)} chiều)")
print(f"   - Tổng cộng: {len(hand_indices) + len(body_indices)} indices.")

🚀 Bắt đầu tạo indices...
⚙️ Đang trích xuất tên features...
✅ Tìm thấy 264 features (Khớp 264).
   Sample features: ['Spine_Xposition', 'Spine_Yposition', 'Spine_Zposition', 'Spine1_Xposition', 'Spine1_Yposition']

🔍 Đang phân loại...

💾 Đã lưu thành công:
   - /home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT_indices/hand_indices.npy (174 chiều)
   - /home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT_indices/body_indices.npy (90 chiều)
   - Tổng cộng: 264 indices.


In [9]:
import os
import sys
import torch
import numpy as np
import joblib

# # 1. Setup Paths
# ROOT = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# if ROOT not in sys.path:
#     sys.path.insert(0, ROOT)

from models.vq_diffusion import create_vq_latent_diffusion

# ==========================================
# CẤU HÌNH KIỂM TRA
# ==========================================
class Config:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Đường dẫn quan trọng
    checkpoints_dir = "./checkpoints"
    dataset_name = 'beat'
    vqvae_name = 'VQVAE_BEAT'
    
    # File motion thật để test (Chọn 1 file bất kỳ trong tập train/val)
    # File này phải CHƯA qua xử lý (Raw .npy 264 chiều)
    test_motion_path = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT_numpy/npy/1/1_wayne_0_1_1_sentence_000.npy" 
    
    # Load lại Mean/Std/Indices
    stats_path = "./global_pipeline.pkl"
    hand_indices_path = "./datasets/BEAT_indices/hand_indices.npy"
    body_indices_path = "./datasets/BEAT_indices/body_indices.npy"

args = Config()

print("🚀 BẮT ĐẦU SANITY CHECK VQ-VAE...")

# ==========================================
# 2. LOAD TÀI NGUYÊN
# ==========================================
# A. Load Stats (Mean/Std) để Normalize
if not os.path.exists(args.stats_path):
    print(f"❌ Thiếu file stats: {args.stats_path}")
    sys.exit(1)

pipeline = joblib.load(args.stats_path)
scaler = pipeline.named_steps['stdscale']
mean = torch.from_numpy(scaler.data_mean_).float().to(args.device)
std = torch.from_numpy(scaler.data_std_).float().to(args.device)

# B. Load Indices (Để đo lỗi riêng biệt)
if os.path.exists(args.hand_indices_path):
    hand_indices = torch.from_numpy(np.load(args.hand_indices_path)).long().to(args.device)
    body_indices = torch.from_numpy(np.load(args.body_indices_path)).long().to(args.device)
    print("✅ Đã load Hand/Body Indices.")
else:
    print("⚠️ Không tìm thấy Indices, sẽ chỉ tính lỗi tổng.")
    hand_indices = None

# C. Load Model (VQ-VAE nằm bên trong)
print("📦 Loading Model...")
model = create_vq_latent_diffusion(
    dataset_name=args.dataset_name,
    vqvae_name=args.vqvae_name,
    checkpoints_dir=args.checkpoints_dir,
    device=args.device,
    freeze_vqvae=True, # Chỉ test, không train
    scale_factor=1.0   # Scale không quan trọng khi test recon (nhân rồi chia sẽ hết)
)
model.eval()

# ==========================================
# 3. CHẠY KIỂM TRA (RECONSTRUCTION) - FIXED
# ==========================================
# Load motion thật
if not os.path.exists(args.test_motion_path):
    print(f"❌ Không tìm thấy file motion: {args.test_motion_path}")
    sys.exit(1)

raw_np = np.load(args.test_motion_path)
if raw_np.ndim == 2:
    raw_np = raw_np[np.newaxis, ...] # (1, T, 264)

# [FIX 1] Padding cho chia hết số nén (Downsampling factor = 8 với BEAT)
# 164 % 8 != 0 -> Gây lỗi Conv1D
B, T, D = raw_np.shape
down_t = 3 # Config của BEAT
stride = 2 ** down_t # = 8
remainder = T % stride

if remainder != 0:
    pad_len = stride - remainder
    # Padding thêm vào cuối chuỗi thời gian bằng cách lặp lại frame cuối (edge)
    raw_np = np.pad(raw_np, ((0,0), (0, pad_len), (0,0)), mode='edge')
    print(f"⚠️ Đã pad dữ liệu từ {T} -> {raw_np.shape[1]} frames (cho chia hết {stride})")

# Chuyển sang Tensor & Normalize
real_motion = torch.from_numpy(raw_np).float().to(args.device)
real_motion_norm = (real_motion - mean) / std

print(f"\n🎬 Input Shape (Normalized & Padded): {real_motion_norm.shape}")

# [FIX 2] Gọi trực tiếp VQ-VAE để tránh Assert độ dài của Diffusion Wrapper
with torch.no_grad():
    # A. Encode trực tiếp bằng VQ-VAE
    # Hàm encode trả về: code_idx, all_codes (Q, B, D, T_latent)
    code_idx, all_codes = model.vqvae.encode(real_motion_norm)
    
    # B. Lấy Latent Sum (Continuous)
    # Shape lúc này là (B, D, T_latent) - đúng chuẩn để đưa vào Decoder
    latent_sum = all_codes.sum(dim=0) 
    
    # C. Decode trực tiếp
    # Không dùng model.decode_from_latent vì nó yêu cầu shape (B, T, D) và scale_factor
    # Ta đưa thẳng vào decoder gốc của VQ-VAE
    recon_motion_norm = model.vqvae.decoder(latent_sum)

    # Cắt bỏ phần padding thừa lúc nãy (nếu muốn chính xác tuyệt đối)
    if remainder != 0:
        recon_motion_norm = recon_motion_norm[:, :T, :]
        real_motion_norm = real_motion_norm[:, :T, :]

# ==========================================
# 4. TÍNH TOÁN SAI SỐ (ERROR METRICS)
# ==========================================
# Tính MSE trên không gian Normalized
diff = (real_motion_norm - recon_motion_norm) ** 2
mse_total = diff.mean().item()

print("\n📊 KẾT QUẢ ĐÁNH GIÁ VQ-VAE:")
print(f"🔹 MSE Tổng thể (Càng thấp càng tốt): {mse_total:.6f}")

if hand_indices is not None:
    # Tính MSE riêng cho tay
    mse_hand = diff[:, :, hand_indices].mean().item()
    mse_body = diff[:, :, body_indices].mean().item()
    
    print(f"🔹 MSE Body: {mse_body:.6f}")
    print(f"🔹 MSE Hand: {mse_hand:.6f}")
    
    ratio = mse_hand / mse_body if mse_body > 0 else 0
    print(f"👉 Tỷ lệ Lỗi Tay / Lỗi Body: {ratio:.2f} lần")
    
    if ratio > 5.0:
        print("\n⚠️  CẢNH BÁO: Lỗi ở tay cao gấp 5 lần body!")
        print("   -> CHÍNH XÁC: VQ-VAE đang nén mất chi tiết tay.")
        print("   -> Lời khuyên: Đừng cố chỉnh Diffusion nữa. VQ-VAE là thủ phạm.")
    elif mse_hand < 0.1: 
        print("\n✅ VQ-VAE tái tạo tay TỐT.")
        print("   -> Vấn đề nằm ở Diffusion Model chưa học được hoặc mất Gradient.")
    else:
        print("\n⚠️ VQ-VAE tái tạo tay TRUNG BÌNH.")

# ==========================================
# 5. LƯU KẾT QUẢ
# ==========================================
recon_motion = recon_motion_norm * std + mean
recon_np = recon_motion.cpu().numpy().squeeze()

save_path = "vqvae_sanity_check.npy"
np.save(save_path, recon_np)
print(f"\n💾 Đã lưu file tái tạo tại: {save_path}")
print(f"   Hãy visualize file này. Nếu tay xấu -> 100% lỗi do VQ-VAE.")

🚀 BẮT ĐẦU SANITY CHECK VQ-VAE...
✅ Đã load Hand/Body Indices.
📦 Loading Model...
[INFO] Using scale_factor=1.000000 for latent normalization
Loading VQ-VAE from ./checkpoints/beat/VQVAE_BEAT/model/best_model.tar
VQ-VAE loaded successfully
VQ-VAE frozen
VQLatentDiffusion initialized:
  - Input features: 512
  - Latent seq length: 45
  - Transformer latent dim: 512
  - Scale factor: 1.000000
⚠️ Đã pad dữ liệu từ 164 -> 168 frames (cho chia hết 8)

🎬 Input Shape (Normalized & Padded): torch.Size([1, 168, 264])

📊 KẾT QUẢ ĐÁNH GIÁ VQ-VAE:
🔹 MSE Tổng thể (Càng thấp càng tốt): 0.018086
🔹 MSE Body: 0.050603
🔹 MSE Hand: 0.001267
👉 Tỷ lệ Lỗi Tay / Lỗi Body: 0.03 lần

✅ VQ-VAE tái tạo tay TỐT.
   -> Vấn đề nằm ở Diffusion Model chưa học được hoặc mất Gradient.

💾 Đã lưu file tái tạo tại: vqvae_sanity_check.npy
   Hãy visualize file này. Nếu tay xấu -> 100% lỗi do VQ-VAE.
